In [66]:
import tweepy
import os
from dotenv import load_dotenv
import datetime as dt
import re

In [318]:
load_dotenv()
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_TOKEN_KEY')
access_token_secret = os.getenv('TWITTER_TOKEN_SECRET')

In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

In [11]:
api = tweepy.API(auth)

In [38]:
screen_name="VaxHuntersCAN"

In [426]:
tweets = api.user_timeline(screen_name, include_rts=True, count=200, tweet_mode = 'extended')

In [427]:
now = dt.datetime.now()
time_36h = now - dt.timedelta(hours=36)

In [428]:
# extract tweets from last 36hrs
tweets_36h = []
for tweet in tweets:
    if tweet.created_at > time_36h:
        tweets_36h.append(tweet)
    else:
        print('Old tweet')

In [429]:
# number tweets from last 36h
len(tweets_36h)

200

In [154]:
full_text = "[ON] Richmond Hill 40+ APPOINTMENTS FROM 10am-5:30pm\nShoppers Drug Mart - Oakridges: 13070 Yonge Street\n22 AstraZeneca doses available, Online Booking"
lines = full_text.split("\n")
age_pat = re.compile(r"(street|road|drive|boulevard|avenue)", re.IGNORECASE)
for line in lines:
    age = age_pat.search(line)
    if age:
        street = age.group(0)
        sub = st_lookup[street.lower()]
        line = re.sub(street, sub, line)
        print(line)

Shoppers Drug Mart - Oakridges: 13070 Yonge st


In [153]:
st_lookup = {'street': 'st', 'road': 'rd', 'drive': 'dr', 'boulevard': 'blvd', 'avenue':'ave'}

In [189]:
a = []
if(a):
    print(a[0])

In [340]:
extract.age_pat.findall("18 19")

[]

In [458]:
class TweetExtract(object):
    
    def __init__(self):
        self.age_pat = re.compile(r"(\d{2})\+")
        self.year_pat = re.compile(r"(Born|born).*(19\d{2})")
        self.postal_pat = re.compile(r"[A-Z]\d[A-Z]")
        self.address_pat = re.compile(r".*(\d+.*\s(?:st|rd|dr|ave|blvd|hwy|edge))(\s|\W|\b)", re.IGNORECASE)
        self.sub_addr_pat = re.compile(r"(?:\d+\s*[\-]*)+(?:\s\w+)*\s(?:st|rd|dr|ave|blvd|hwy)(?:(?:[\s\.]\d+)|(?:,(?:\s*\w+)*)+)?", re.IGNORECASE)
        self.street_pat = re.compile(r"(street|road|drive|boulevard|avenue|highway)", re.IGNORECASE)
        self.st_lookup = {'street': 'st', 'road': 'rd', 'drive': 'dr', 'boulevard': 'blvd', 'avenue':'ave', 'highway':'hwy'}
        self.t_link_pat = re.compile(r"https:\/\/t.co\/\w+")
        self.province_pat = re.compile(r"\[(\w{2})\]")
        self.all_postal_pat = re.compile(r'\"([A-Z])\".*(?:HOTSPOT|hotspot|Hotspot|hot spot)')
        self.num_match = 0
        
        self.API_KEY = os.getenv('GOOGLE_API_KEY')
        
        # load hotspot postals
        self.hotspot_postals()
        
        # load postal codes and ANN index
        self.tree = pickle.load(open('./utils/ball_postal_index.p', "rb"))
        postal_codes = pd.read_csv('./CanadianPostalCodes.csv')
        self._fsa = np.asarray(postal_codes.loc[:, ['FSA']])
        self._radius = [self.km_to_rad(2.5)] # search in 2.5km radius
        del postal_codes
        
    def hotspot_postals(self):
        resp = requests.get('http://localhost:3000/api/clinics/postal')
        if resp.status_code == 200:
            try:
                codes = json.loads(resp.text)
                if(codes):
                    self.hotspot_map = codes['mapped']
            except Exception as e:
                self.hotspot_map = {}
                pass # do nothing
        
    def get_age(self, full_text):
        extracted_age = None
        ages = self.age_pat.findall(full_text)
        year = self.year_pat.search(full_text)
        
        if(ages): # check for age
            min_age = ages[0]
            for age in ages:
                if age < min_age:
                    min_age = age
            extracted_age = min_age
        elif (year): # check for year if age not included
            extracted_age = 2021 - int(year.group(2))
            
        return extracted_age
    
    def nominatim(self,address):
        lat = lon = None
        # create parameter string to minimize formatting issues
        payload= {'q': address, 'format': 'jsonv2','countrycodes':'ca','addressdetails':1}
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        resp = requests.get('https://nominatim.openstreetmap.org/search',params=payload_str)
        if resp.status_code == 200:
            try:
                geodata = json.loads(resp.text)
                if(geodata):
                    lat = geodata[0]['lat']
                    lon = geodata[0]['lon']
            except Exception as e:
                pass # do nothing
        return lat, lon
    
    def gmaps(self,address):
        lat = lon = None
        payload= {'address': address,'components':'country:CA', 'key':self.API_KEY}
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        resp = requests.get('https://maps.googleapis.com/maps/api/geocode/json',params=payload_str)
        if resp.status_code == 200:
            try:
                geodata = json.loads(resp.text)
                if(geodata):
                    lat = geodata["results"][0]["geometry"]["location"]['lat']
                    lon = geodata["results"][0]["geometry"]["location"]['lng']
            except Exception as e:
                pass # do nothing
        return lat, lon
    
    def search_address(self, address, sub_addr=None):
#         n_lat, n_lon = self.nominatim(address)
        g_lat, g_lon = self.gmaps(address)
        
        lat = g_lat #or n_lat
        lon = g_lon #or n_lon
        
        if(not lat and sub_addr):
            return self.search_address(sub_addr)
        
#         if(g_lat and n_lat):
#             print(f'Diff in lat: {(float(g_lat) - float(n_lat))/float(g_lat)*100}')
#             print(f'Diff in lon: {(float(g_lon) - float(n_lon))/float(g_lon) * 100}')
        return lat, lon
    
    # convert radians to km
    def rad_to_km(self,rad):
        r = 6371 # earth radius in km
        return rad*r # radians = arc_len / radius
    
    def km_to_rad(self,km):
        r = 6371 # earth radius in km
        return km/r # radians = arc_len / radius
    
    def deg_to_rad(self,x): # convert lat, lon pair from deg to radians
        return [float(x[0]) * np.pi / 180, float(x[1]) * np.pi / 180]
    
    def find_postals(self, lat, lon):
        point_rad = [np.array(self.deg_to_rad([lat, lon]))]
        ind = self.tree.query_radius(point_rad, r=self._radius)
        postals = np.unique(self._fsa[ind[-1]])
        return postals
        
    def get_postal(self, full_text):
        extracted_postal = []
        clean_text = self.t_link_pat.sub('', full_text)
        postals = self.postal_pat.findall(clean_text)
        all_postal = self.all_postal_pat.search(full_text)
        if all_postal:
            letter = all_postal.group(1)
            extracted_postal = self.hotspot_map.get(letter, [])
        elif postals: #check for postal code
            extracted_postal = postals
        else: # check for address
            lines = clean_text.split("\n")
            for line in lines:
                # replace street endings to standardize
                st_match = self.street_pat.search(line)
                if st_match:
                    street = st_match.group(0)
                    sub = self.st_lookup[street.lower()]
                    line = re.sub(street, sub, line)
               
                # search for address
                match = self.sub_addr_pat.search(line)
                if match:
                    addr = match.group(0)
                    prov = self.province_pat.search(full_text) # extract province
                    if prov:
                        full_addr = f'{addr}, {prov.group(1)}'
                    else:
                        full_addr = addr
                    lat,lon = self.search_address(full_addr)
                    
                    if(lat):                        
                        # extract nearby postal codes
                        extracted_postal = self.find_postals(lat,lon)
        
        return extracted_postal
        
    def single_tweet_extract(self, tweet):
        t_data = dict()
        
        full_text = tweet.full_text

        age = self.get_age(full_text)
        postal_code = self.get_postal(full_text)

        if(age and len(postal_code) > 0):
            if len(tweet.entities['urls']) > 0:
                url = tweet.entities['urls'][0]['expanded_url']
            else:
                url = None

            t_data['id'] = tweet.id # use id for max_id
            t_data['name'] = tweet.user.name # Vaccine Hunters Canada
            t_data['screen_name'] = tweet.user.screen_name
            t_data['lastUpdated'] = tweet.created_at
            t_data['eligibility'] = full_text
            t_data['postal_code'] = postal_code
            t_data['status'] = "open"
            t_data['age'] = age
            t_data['booking_link'] = url
        elif(hasattr(tweet, 'quoted_status')):
            return self.single_tweet_extract(tweet.quoted_status)

        return t_data
    
    def extract_tweets(self, tweets):
        valid_tweet_data = []
        for tweet in tweets:
            tweet_data = self.single_tweet_extract(tweet)
            if(tweet_data):
                valid_tweet_data.append(tweet_data)
#         print(self.num_match)        
        return valid_tweet_data

In [460]:
extract = TweetExtract()
data = extract.extract_tweets(tweets_36h)

[ON] NEW POPUP ALERT: St. John the Evangelist Catholic School (23 George Street) is open to 18+ living or working in ALL "M" HOTSPOTS. 10am-9pm TODAY!  Let's get vaxxed!! https://t.co/A4FzWCxNHm #COVID19 #COVID19Vaccine #COVID19ON #vhcON
M
['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2J', 'M2M', 'M2R', 'M3A', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4H', 'M4X', 'M5A', 'M5B', 'M5N', 'M5V', 'M6A', 'M6B', 'M6E', 'M6H', 'M6K', 'M6L', 'M6M', 'M6N', 'M8V', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W']


In [435]:
tweets_df = pd.DataFrame(data)

In [436]:
len(tweets_df)

59

In [387]:
tweets_df.head(3)

,id,name,screen_name,lastUpdated,eligibility,postal_code,status,age,booking_link
0,1388795422991396869,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 10:00:06,[ON] 18+ RESIDENT OR WORKER IN M3J M3K M3M M3N...,"[M3J, M3K, M3M, M3N, M6M, M9L, M9M, M9N]",open,18,None
1,1388697630205751297,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 03:31:30,[ON] Brampton 40+ \nShoppers Drug Mart 10665 B...,"[L6R, L6S, L6T, L6Z, L7A, L7C]",open,40,None
2,1388689726316679169,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 03:00:06,[ON] 18+ RESIDENT OR WORKER IN M3J M3K M3M M3N...,"[M3J, M3K, M3M, M3N, M6M, M9L, M9M, M9N]",open,18,None


In [393]:
tweets_df.sort_values(by="id", ascending=False)

,id,name,screen_name,lastUpdated,eligibility,postal_code,status,age,booking_link
0,1388795422991396869,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 10:00:06,[ON] 18+ RESIDENT OR WORKER IN M3J M3K M3M M3N...,"[M3J, M3K, M3M, M3N, M6M, M9L, M9M, M9N]",open,18,None
1,1388697630205751297,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 03:31:30,[ON] Brampton 40+ \nShoppers Drug Mart 10665 B...,"[L6R, L6S, L6T, L6Z, L7A, L7C]",open,40,None
2,1388689726316679169,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 03:00:06,[ON] 18+ RESIDENT OR WORKER IN M3J M3K M3M M3N...,"[M3J, M3K, M3M, M3N, M6M, M9L, M9M, M9N]",open,18,None
3,1388673506188070913,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 01:55:38,[ON] TORONTO - Vaughan 40+ \nShoppers Drug Mar...,"[L3P, L3R, L3S, L3T, L4G, L6B, L6C, L6E]",open,40,https://outlook.office365.com/owa/calendar/Sho...
4,1388672753344159744,Vaccine Hunters Canada,VaxHuntersCan,2021-05-02 01:52:39,[ON] Brampton 40+\nShoppers Drug Mart 1326 (HW...,"[L0P, L4H, L4L, L6P, L6T, L6Z]",open,40,https://outlook.office365.com/owa/calendar/Sho...
...,...,...,...,...,...,...,...,...,...
85,1388258685357539333,Vaccine Hunters Canada,VaxHuntersCan,2021-04-30 22:27:17,[ON] Thornhill 40+\nShoppers Drug Mart Spring ...,"[L3P, L3R, L3T, L4C, L4J, L5T, L6B, L6R, L9L, ...",open,40,https://outlook.office365.com/owa/calendar/Sho...
86,1388251036515700737,Vaccine Hunters Canada,VaxHuntersCan,2021-04-30 21:56:54,[BC] Penticton Born 1991/before (30+)\nShopper...,[V2A],open,30,None
87,1388249943673032711,Vaccine Hunters Canada,VaxHuntersCan,2021-04-30 21:52:33,"[ON] 18+ PEEL HOTSPOTS, M8V, M9A, M9B, M9C ETO...","[M8V, M9A, M9B, M9C]",open,18,https://thp-public.vertoengage.com/
88,1388237128572801024,Vaccine Hunters Canada,VaxHuntersCan,2021-04-30 21:01:38,[ON] London 40+ Walkin \nShoppers 736\n431 Ric...,"[N5W, N5X, N5Y, N5Z, N6A, N6B, N6C, N6E, N6G, ...",open,40,None


In [437]:
# load to API
api_key = os.getenv('API_KEY')
secret = os.getenv('SECRET_KEY')
api_url = "https://findavaccine.ca" #"http://localhost:3000"

In [438]:
# use API to update
notified = 0
for i, row in tweets_df.iterrows():
    payload = {"notify": True, **row}
    print(f'Updating {row["name"]}')
    resp = requests.post(f'{api_url}/api/tweets/update', data=payload, auth=(api_key, secret) )
    status = resp.status_code
    if status == 200:
        r = json.loads(resp.text)
        notified += r["notify"]
    else:
        print(f'Request failed for {row["id"]}: {resp.text}')

Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating Vaccine Hunters Canada
Updating

In [439]:
print(f'Notified {notified} users')

Notified 40 users


In [401]:
# get latest id
resp = requests.get(f'{api_url}/api/tweets/info', auth=(api_key, secret) )
status = resp.status_code
if status == 200:
    r = json.loads(resp.text)
    print(r)

{'twitters': [{'screen_name': 'VaxHuntersCan', 'name': 'Vaccine Hunters Canada', 'since_id': '1389897683196841984'}]}


In [400]:
r['twitters'][0]["since_id"] == "1388795422991396869"

True